In [48]:
import pandas as pd
df = pd.read_csv('parsed_shot_events.csv')
df

,season,gameId,eventId,period,timeInPeriod,eventType,teamId,shooter,goalie,HomevsAway,shotType,emptyNetAway,emptyNetHome,powerplayHome,powerplayAway,coordinates,result
0,20162017,2016020001,8,1,01:11,shot-on-goal,10,Mitch Marner,Craig Anderson,away,wrist,False,False,False,False,"(-77, 5)",no goal
1,20162017,2016020001,11,1,02:53,shot-on-goal,9,Chris Kelly,Frederik Andersen,home,wrist,False,False,False,False,"(86, 13)",no goal
2,20162017,2016020001,15,1,04:01,shot-on-goal,9,Cody Ceci,Frederik Andersen,home,wrist,False,False,False,False,"(23, -38)",no goal
3,20162017,2016020001,16,1,04:46,shot-on-goal,9,Erik Karlsson,Frederik Andersen,home,slap,False,False,False,False,"(33, -15)",no goal
4,20162017,2016020001,24,1,06:46,shot-on-goal,10,Martin Marincin,Craig Anderson,away,wrist,False,False,False,False,"(-34, 28)",no goal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385071,20202021,2020030415,692,3,16:19,shot-on-goal,14,Blake Coleman,Carey Price,home,wrist,False,False,False,False,"(57, 30)",no goal
385072,20202021,2020030415,693,3,17:13,shot-on-goal,14,Mikhail Sergachev,Carey Price,home,backhand,False,False,False,False,"(56, -31)",no goal
385073,20202021,2020030415,697,3,19:01,shot-on-goal,8,Tyler Toffoli,Andrei Vasilevskiy,away,wrist,True,False,False,True,"(-45, -14)",no goal
385074,20202021,2020030415,698,3,19:10,shot-on-goal,8,Cole Caufield,Andrei Vasilevskiy,away,backhand,True,False,False,True,"(-77, -3)",no goal


In [49]:
import math
import pandas as pd
import ast

def add_distance_angle_column(df):
    def calculate_distance_angle(row):
        # Parse coordinates as a tuple if stored as a string
        coordinates = row['coordinates']
        if isinstance(coordinates, str):
            try:
                coordinates = ast.literal_eval(coordinates)
            except (ValueError, SyntaxError):
                return None, None  # Return None for both if parsing fails
        
        # Ensure coordinates is a tuple with two real values
        if isinstance(coordinates, tuple) and len(coordinates) == 2:
            x, y = coordinates
            if x is None or y is None:
                return None, None  # Return None for both if any coordinate value is None
        else:
            return None, None  # Return None for both if coordinates are missing or invalid
        
        # Determine the reference point based on the conditions
        if (row['period'] % 2 == 1 and row['HomevsAway'] == 'away') or (row['period'] % 2 == 0 and row['HomevsAway'] == 'home'):
            reference_point = (-90, 0)
        else:
            reference_point = (90, 0)
        
        # Calculate distance
        distance = math.dist(reference_point, (x, y))
        
        # Calculate angle (in degrees) between the point and the reference point
        delta_x = x - reference_point[0]
        delta_y = y - reference_point[1]
        angle_radians = math.atan2(delta_y, delta_x)
        angle_degrees = math.degrees(angle_radians)

        if angle_degrees > 90:
           angle_degrees = 180 - angle_degrees

        if angle_degrees < -90 :
           angle_degrees = -180 - angle_degrees

          
        
        
        # Return both distance and angle
        return distance, angle_degrees
    
    # Apply the function to each row in the DataFrame and create the new columns
    df[['distance', 'angle']] = df.apply(calculate_distance_angle, axis=1, result_type='expand')
    
    return df

# Exemple d'utilisation
df = add_distance_angle_column(df)


In [50]:
def add_emptyNet(df):
  def emptyNet(row):
    if (row['HomevsAway'] == 'away' and row['emptyNetHome']) or (row['HomevsAway'] == 'home' and row['emptyNetAway']):
        return 1
    else:
        return 0

  df['emptyNet'] = df.apply(emptyNet,axis=1)
  return df

df = add_emptyNet(df)

In [51]:
# Reorder the columns to place 'distance' after 'coordinates'
cols = df.columns.tolist()
coords_index = cols.index('coordinates')
    # Move 'distance' to the next position after 'coordinates'
cols.insert(coords_index + 1, cols.pop(cols.index('distance')))
cols.insert(coords_index + 2, cols.pop(cols.index('angle')))
cols.insert(coords_index + 3, cols.pop(cols.index('emptyNet')))
    
df = df[cols]

df

,season,gameId,eventId,period,timeInPeriod,eventType,teamId,shooter,goalie,HomevsAway,shotType,emptyNetAway,emptyNetHome,powerplayHome,powerplayAway,coordinates,distance,angle,emptyNet,result
0,20162017,2016020001,8,1,01:11,shot-on-goal,10,Mitch Marner,Craig Anderson,away,wrist,False,False,False,False,"(-77, 5)",13.928388,21.037511,0,no goal
1,20162017,2016020001,11,1,02:53,shot-on-goal,9,Chris Kelly,Frederik Andersen,home,wrist,False,False,False,False,"(86, 13)",13.601471,72.897271,0,no goal
2,20162017,2016020001,15,1,04:01,shot-on-goal,9,Cody Ceci,Frederik Andersen,home,wrist,False,False,False,False,"(23, -38)",77.025970,-29.560354,0,no goal
3,20162017,2016020001,16,1,04:46,shot-on-goal,9,Erik Karlsson,Frederik Andersen,home,slap,False,False,False,False,"(33, -15)",58.940648,-14.743563,0,no goal
4,20162017,2016020001,24,1,06:46,shot-on-goal,10,Martin Marincin,Craig Anderson,away,wrist,False,False,False,False,"(-34, 28)",62.609903,26.565051,0,no goal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385071,20202021,2020030415,692,3,16:19,shot-on-goal,14,Blake Coleman,Carey Price,home,wrist,False,False,False,False,"(57, 30)",44.598206,42.273689,0,no goal
385072,20202021,2020030415,693,3,17:13,shot-on-goal,14,Mikhail Sergachev,Carey Price,home,backhand,False,False,False,False,"(56, -31)",46.010868,-42.357455,0,no goal
385073,20202021,2020030415,697,3,19:01,shot-on-goal,8,Tyler Toffoli,Andrei Vasilevskiy,away,wrist,True,False,False,True,"(-45, -14)",47.127487,-17.281498,0,no goal
385074,20202021,2020030415,698,3,19:10,shot-on-goal,8,Cole Caufield,Andrei Vasilevskiy,away,backhand,True,False,False,True,"(-77, -3)",13.341664,-12.994617,0,no goal


In [52]:
# Séparer les données en ensembles d'entraînement et de test
df_train = df[df['season'] != 20202021]  # Données d'entraînement pour toutes les saisons sauf 2020-2021
df_test = df[df['season'] == 20202021]   # Données de test uniquement pour la saison 2020-2021

# Vérification des dimensions
print("Taille de l'ensemble d'entraînement :", df_train.shape)
print("Taille de l'ensemble de test :", df_test.shape)

Taille de l'ensemble d'entraînement : (327342, 20)
Taille de l'ensemble de test : (57734, 20)


In [53]:
# Remplacer 'no goal' par 0 et 'goal' par 1 dans la colonne 'result'
df_train['result'] = df_train['result'].replace({'no goal': 0, 'goal': 1})
df_test['result'] = df_test['result'].replace({'no goal': 0, 'goal': 1})
# Garder uniquement les colonnes sélectionnées
df_train = df_train[['distance', 'angle', 'emptyNet', 'result']]
df_test = df_test[['distance', 'angle', 'emptyNet', 'result']]



C:\Users\Admin\AppData\Local\Temp\ipykernel_41620\63657681.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['result'] = df_train['result'].replace({'no goal': 0, 'goal': 1})
C:\Users\Admin\AppData\Local\Temp\ipykernel_41620\63657681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['result'] = df_train['result'].replace({'no goal': 0, 'goal': 1})
C:\Users\Admin\AppData\Local\Temp\ipykernel_41620\63657681.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a

In [56]:
df_train.to_csv('train_data.csv',index=False)
df_train

,distance,angle,emptyNet,result
0,13.928388,21.037511,0,0
1,13.601471,72.897271,0,0
2,77.025970,-29.560354,0,0
3,58.940648,-14.743563,0,0
4,62.609903,26.565051,0,0
...,...,...,...,...
327337,39.204592,-37.746805,0,0
327338,9.486833,18.434949,0,0
327339,56.320511,-6.115504,0,0
327340,59.841457,9.619728,0,0


In [57]:
df_test.to_csv('test_data.csv',index=False)
df_test

,distance,angle,emptyNet,result
327342,166.544288,10.027909,0,0
327343,141.230308,-10.195988,0,0
327344,145.344419,-12.315018,0,0
327345,138.600144,16.342883,0,0
327346,143.223601,3.202021,0,0
...,...,...,...,...
385071,44.598206,42.273689,0,0
385072,46.010868,-42.357455,0,0
385073,47.127487,-17.281498,0,0
385074,13.341664,-12.994617,0,0
